In [ ]:
import numpy as np
import pandas as pd
import mplfinance as mpf

stocks = pd.read_csv('stocks_1312.csv')

stocks.iloc[:, 0] = pd.to_datetime(stocks.iloc[:, 0], format = "%Y/%m/%d")
stocks = stocks.set_index(pd.DatetimeIndex(stocks["Date"]))

stocks = stocks.tail(150)

colorSet = mpf.make_marketcolors(up = 'r', down = 'g')

styleSet = mpf.make_mpf_style(marketcolors = colorSet)

kwargs = dict(type = 'candle', style = styleSet, figscale = 2.5)

#KD K為快線、D為慢線
stocks_close = [float(line) for line in stocks['Close']] #收盤價
stocks_high = [float(line) for line in stocks['High']]   #最高價
stocks_low = [float(line) for line in stocks['Low']]     #最低價
rsv = []                                                 #RSV list
Kpoint = [50]                                            #K值 list
Dpoint = [50]                                            #D值 list

#最近12天的 KD指標
#計算 RSV
for i in range(11, len(stocks['Date'])):
    low = min(stocks_low[i-11 : i+1])
    high = max(stocks_high[i-11 : i+1])
    rsv.append(round(100 * ((stocks_close[i] - low) / (high - low)), 2))
    
#計算K值和 D值
for i in range(0, len(rsv) - 1):
    Kpoint.append(round((Kpoint[i] * (2 / 3) + rsv[i + 1] * (1 / 3)), 2))
    Dpoint.append(round((Dpoint[i] * (2 / 3) + Kpoint[i + 1] * (1 / 3)), 2))
    
#補 0
for i in range(0, 11):
    Kpoint.insert(i, np.nan)
    Dpoint.insert(i, np.nan)

################################################################################################################################
#策略實現

#繪製點所使用的串列 (初始化) (for上面的圖)
points_buy = [np.nan] * len(stocks)
points_sell = [np.nan] * len(stocks)

#繪製點所使用的串列 (初始化) (for下面的圖)
p_buy = [np.nan] * len(stocks)
p_sell = [np.nan] * len(stocks)

for i in range(1, len(Kpoint)):
    #買入點 (如果 k值小於20 且 K線突破 D線 ) * 0.98 方便觀看
    if(Kpoint [i] < 20 and Kpoint[i - 1] < Dpoint[i - 1] and Kpoint[i] > Dpoint[i]):
        points_buy[i] = stocks_low[i] * 0.98
        p_buy[i] = Kpoint[i]
    #賣出點 (如果k值大於80 且 k線跌破 D線 3百分比) * 1.02 方便觀看
    elif(Kpoint [i] > 80 and Kpoint[i - 1] > Dpoint[i - 1] and Kpoint[i] < Dpoint[i]):
        points_sell[i] = stocks_high[i] * 1.02
        p_sell[i] = Kpoint[i]
        
#點狀圖 type = 'scatter'，markersize可以更改 marker大小
draw = [
        mpf.make_addplot(points_buy, type = 'scatter', marker = '^', markersize = 100, color = 'blue'),
        mpf.make_addplot(points_sell, type = 'scatter', marker = 'v', markersize = 100, color = 'darkorange'),
        mpf.make_addplot(Kpoint, panel = 1, color = 'red'), 
        mpf.make_addplot(Dpoint, panel = 1, color = 'blue'),
        mpf.make_addplot(p_buy, type = 'scatter', panel = 1, color = 'blue', markersize = 50),
        mpf.make_addplot(p_sell, type = 'scatter', panel = 1, color = 'darkorange', markersize = 50)
       ]

mpf.plot(stocks, **kwargs, addplot = draw, panel_ratios = (1.5, 0.8))